In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from arch import arch_model

import os
import sys
from dotenv import load_dotenv

load_dotenv()
REPO_PATH: str = os.getenv('REPO_PATH')

sys.path.insert(0, rf'{REPO_PATH}src_HF')
from utils import *

#### Realized volatility

$$
\text{Realized Volatility}=\sqrt{\sum_{i=1}^{n}(r_i)^2}
$$

In [ ]:
FUTURE = 'CLc1'

df = pd.read_csv(rf'{REPO_PATH}data\time_series\{FUTURE}_High_Frequency.csv')

df.index = pd.to_datetime(df['Date'])

df['LOGRET'] = np.log(df['CLOSE']).diff()

df.loc[df.index[0], 'LOGRET'] = 0

display(df)

RESAMPLE_WINDOW: str = '5min'

df_5min = pd.DataFrame(
    {
        'CLOSE': df['CLOSE'].resample(RESAMPLE_WINDOW).last(),
        'VOLUME': df['VOLUME'].resample(RESAMPLE_WINDOW).sum(),
        'COUNT': df['COUNT'].resample(RESAMPLE_WINDOW).sum(),
        'LOGRET': df['LOGRET'].resample(RESAMPLE_WINDOW).sum(),
        'REALIZED_VOL': np.sqrt((df['LOGRET'] ** 2).resample(RESAMPLE_WINDOW).sum())
    }
)

display(df_5min)

### Add temporal features

In [ ]:

df_5min['HOUR'] = df_5min.index.hour
df_5min['MINUTE'] = df_5min.index.minute
df_5min['MINUTE_OF_DAY'] = df_5min['HOUR'] * 60 + df_5min['MINUTE']
df_5min['DAY_OF_WEEK'] = df_5min.index.dayofweek
df_5min['DAY_OF_MONTH'] = df_5min.index.day

display(df_5min)

### Save data

In [ ]:
df_5min.to_csv(rf'{REPO_PATH}data\time_series\{FUTURE}_5min_processed.csv')